In [1]:
import os
import json
import csv
from typing import List, Dict

def extract_eval_data(data_dir: str = "nameguess/data") -> List[Dict]:
    """Extract evaluation dataset metadata from JSON files"""
    results = []
    
    # Identify evaluation dataset files
    eval_files = [f for f in os.listdir(data_dir) 
                 if f.endswith(".json") and "eval" in f.lower()]
    
    for eval_file in eval_files:
        file_path = os.path.join(data_dir, eval_file)
        
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            for entry in data:
                results.append({
                    "dataset_name": eval_file,
                    "table_id": entry.get("table_id", ""),
                    "gt_label": entry.get("gt_label", ""),
                    "technical_name": entry.get("technical_name", ""),
                    "difficulty": entry.get("difficulty", "unknown")
                })
    
    return results

def write_to_csv(data: List[Dict], output_file: str = "nameguess_metadata.csv"):
    """Write extracted data to CSV file"""
    fieldnames = ["dataset_name", "table_id", "gt_label", "technical_name", "difficulty"]
    
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    # Clone repository first if needed
    if not os.path.exists("nameguess/data"):
        raise FileNotFoundError("""
        NameGuess data directory not found. 
        Clone the repository first:
        git clone https://github.com/amazon-science/nameguess.git
        """)
    
    extracted_data = extract_eval_data()
    write_to_csv(extracted_data)
    print(f"Successfully created metadata CSV with {len(extracted_data)} entries")


Successfully created metadata CSV with 1376 entries


In [1]:
import csv
import ast

def process_csv(input_csv: str, output_csv: str):
    """
    Processes the input CSV to extract pairs of gt_label and technical_name items.
    Writes a new CSV where each row contains a full column description and its abbreviation.
    """
    mapping_rows = []

    with open(input_csv, newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            # The CSV stores the lists as strings; safely convert them to Python lists.
            gt_label_str = row['gt_label']
            tech_name_str = row['technical_name']
            
            try:
                gt_labels = ast.literal_eval(gt_label_str)
                tech_names = ast.literal_eval(tech_name_str)
            except Exception as e:
                print(f"Error parsing lists in row with table_id {row.get('table_id', 'N/A')}: {e}")
                continue

            if len(gt_labels) != len(tech_names):
                print(f"Length mismatch in row with table_id {row.get('table_id', 'N/A')}. Skipping this row.")
                continue

            # Append each pair from the lists.
            for full, abbr in zip(gt_labels, tech_names):
                mapping_rows.append([full, abbr])
    
    # Write the extracted pairs to a new CSV file.
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Optionally, include a header row. If not needed, comment out the next line.
        writer.writerow(["Full Description", "Abbreviation"])
        writer.writerows(mapping_rows)

    print(f"Successfully processed {len(mapping_rows)} pairs into '{output_csv}'.")

if __name__ == "__main__":
    # Update these file names if needed.
    input_csv = "nameguess_metadata.csv"
    output_csv = "mapping.csv"
    process_csv(input_csv, output_csv)


Successfully processed 9218 pairs into 'mapping.csv'.


In [2]:
import csv
import ast

def process_csv(input_csv: str, output_csv: str):
    """
    Processes the input CSV to extract pairs of gt_label and technical_name items,
    removes duplicates, and sorts them first by abbreviation (second column) and then by full description (first column).
    Writes a new CSV where each row contains a full column description and its abbreviation.
    """
    mapping_set = set()

    # Read the input CSV and extract pairs
    with open(input_csv, newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            # Convert the string representations of lists to actual Python lists.
            gt_label_str = row['gt_label']
            tech_name_str = row['technical_name']
            
            try:
                gt_labels = ast.literal_eval(gt_label_str)
                tech_names = ast.literal_eval(tech_name_str)
            except Exception as e:
                print(f"Error parsing lists in row with table_id {row.get('table_id', 'N/A')}: {e}")
                continue

            if len(gt_labels) != len(tech_names):
                print(f"Length mismatch in row with table_id {row.get('table_id', 'N/A')}. Skipping this row.")
                continue

            # Add each pair to a set to ensure uniqueness.
            for full, abbr in zip(gt_labels, tech_names):
                mapping_set.add((full, abbr))
    
    # Convert the set to a list for sorting.
    mapping_list = list(mapping_set)

    # Sort first by abbreviation (second column) then by full description (first column).
    mapping_list_sorted = sorted(mapping_list, key=lambda pair: (pair[1].lower(), pair[0].lower()))

    # Write the cleaned and sorted pairs to a new CSV file.
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Optionally include a header row.
        writer.writerow(["Full Description", "Abbreviation"])
        writer.writerows(mapping_list_sorted)
    
    print(f"Successfully processed {len(mapping_list_sorted)} unique pairs into '{output_csv}'.")

if __name__ == "__main__":
    input_csv = "nameguess_metadata.csv"  # Change if your CSV has a different name.
    output_csv = "cleaned_mapping.csv"
    process_csv(input_csv, output_csv)


Successfully processed 8036 unique pairs into 'cleaned_mapping.csv'.


In [4]:
import csv
import wordninja

def split_abbreviation(text: str):
    """
    Splits the abbreviation string into words.
    If underscores are present, split on them; otherwise, use wordninja.
    """
    if "_" in text:
        return text.split("_")
    else:
        return wordninja.split(text)

def process_word_comparisons(input_csv: str, output_csv: str):
    """
    Reads the cleaned mapping CSV, breaks each mapping pair into individual word pairs,
    and writes them to a new CSV file with columns: 'Full Word' and 'Abbreviation Word'.
    """
    word_pairs = []

    with open(input_csv, newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            full_desc = row["Full Description"]
            abbr = row["Abbreviation"]

            # Split full description into words (assumes whitespace separation)
            full_words = full_desc.split()
            # Split abbreviation using underscores if available, else use wordninja
            abbr_words = split_abbreviation(abbr)

            if len(full_words) != len(abbr_words):
                print(f"Warning: Word count mismatch for '{full_desc}' and '{abbr}'. Skipping row.")
                continue

            # Append each word pair
            for full_word, abbr_word in zip(full_words, abbr_words):
                word_pairs.append([full_word, abbr_word])

    # Write the word-by-word pairs to a new CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Full Word", "Abbreviation Word"])
        writer.writerows(word_pairs)

    print(f"Successfully processed {len(word_pairs)} word pairs into '{output_csv}'.")

if __name__ == "__main__":
    # Update file names if needed.
    input_csv = "cleaned_mapping.csv"  # This is the file with 8039 (or 9220) items.
    output_csv = "word_by_word_mapping.csv"
    process_word_comparisons(input_csv, output_csv)


Scholastic Assessment Test Grade 11 School' and 'AttainmntSATGrd11Schl'. Skipping row.
Successfully processed 8752 word pairs into 'word_by_word_mapping.csv'.


In [5]:
import csv
import re
import sys

try:
    import wordninja
except ModuleNotFoundError:
    print("Error: The module 'wordninja' is not installed. Please run 'pip install wordninja' and try again.")
    sys.exit(1)

def split_abbreviation(text: str):
    """
    Splits the abbreviation string into words.
    - If underscores are present, split on them.
    - Else if camel-case pattern is detected (uppercase letters after the first character), split based on camel case.
    - Otherwise, use wordninja.
    """
    if "_" in text:
        return text.split("_")
    elif any(c.isupper() for c in text[1:]):  # detects camelCase
        return re.split(r'(?<!^)(?=[A-Z])', text)
    else:
        return wordninja.split(text)

def process_word_comparisons(input_csv: str, output_csv: str):
    """
    Reads the cleaned mapping CSV, breaks each mapping pair into individual word pairs,
    and writes them to a new CSV file with columns: 'Full Word' and 'Abbreviation Word'.
    """
    word_pairs = []

    with open(input_csv, newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            full_desc = row["Full Description"]
            abbr = row["Abbreviation"]

            # Split full description into words (assumes whitespace separation)
            full_words = full_desc.split()
            # Split abbreviation using our updated function
            abbr_words = split_abbreviation(abbr)

            if len(full_words) != len(abbr_words):
                print(f"Warning: Word count mismatch for '{full_desc}' and '{abbr}'. Skipping row.")
                continue

            # Append each word pair
            for full_word, abbr_word in zip(full_words, abbr_words):
                word_pairs.append([full_word, abbr_word])

    # Write the word-by-word pairs to a new CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Full Word", "Abbreviation Word"])
        writer.writerows(word_pairs)

    print(f"Successfully processed {len(word_pairs)} word pairs into '{output_csv}'.")

if __name__ == "__main__":
    input_csv = "cleaned_mapping.csv"  # Update if your file has a different name.
    output_csv = "word_by_word_mapping.csv"
    process_word_comparisons(input_csv, output_csv)


Scholastic Assessment Test Grade 11 School 1' and 'AttainmntSATGr11Schl1'. Skipping row.
Scholastic Assessment Test Grade 11 School' and 'AttainmntSATGrd11Schl'. Skipping row.
scholastic assessment test grade 11 score school' and 'SATGr11ScorSch'. Skipping row.
Successfully processed 12664 word pairs into 'word_by_word_mapping.csv'.


In [13]:
import csv

def process_abbreviations_and_unique(input_csv: str, ambiguous_output_csv: str, unique_output_csv: str):
    """
    Lê o arquivo CSV 'word_by_word_mapping1.csv', converte ambas as colunas para minúsculas,
    e filtra as linhas onde a palavra completa é igual à abreviação ou onde a abreviação tem apenas uma letra.
    
    Agrupa por abreviação para gerar:
      1. Um arquivo de abreviações ambíguas (com mais de uma full word associada),
         incluindo uma terceira coluna com o mapeamento <abbreviation>: <full word mais frequente>.
      2. Um arquivo de palavras únicas – isto é, full words associadas a apenas uma abreviação,
         sendo que essa abreviação não é ambígua (aparece com somente essa full word).
         Nesse arquivo, é também adicionada uma terceira coluna com o mapeamento <abbreviation>: <full word>.
    """
    # Dicionários para agrupamento
    abbr_group = {}   # chave: abreviação, valor: dicionário { full_word: contagem }
    full_group = {}   # chave: full_word, valor: dicionário { abbreviation: contagem }
    
    with open(input_csv, newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            full_word = row["Full Word"].strip().lower()
            abbr_word = row["Abbreviation Word"].strip().lower()
            
            # Ignora se a palavra completa é igual à abreviação
            if full_word == abbr_word:
                continue
            # Ignora abreviações com apenas uma letra
            if len(abbr_word) <= 1:
                continue
            
            # Agrupamento por abreviação
            if abbr_word not in abbr_group:
                abbr_group[abbr_word] = {}
            abbr_group[abbr_word][full_word] = abbr_group[abbr_word].get(full_word, 0) + 1
            
            # Agrupamento por palavra completa
            if full_word not in full_group:
                full_group[full_word] = {}
            full_group[full_word][abbr_word] = full_group[full_word].get(abbr_word, 0) + 1

    # Gera arquivo de abreviações ambíguas: aquelas com mais de uma full word associada.
    ambiguous = {abbr: counts for abbr, counts in abbr_group.items() if len(counts) > 1}
    sorted_ambiguous = sorted(ambiguous.items(), key=lambda x: x[0])
    
    with open(ambiguous_output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Abbreviation", "Full Words (Count)", "Most Frequent Mapping"])
        for abbr, counts in sorted_ambiguous:
            mapping_str = ", ".join([f"{word} ({count})" for word, count in sorted(counts.items())])
            most_freq = max(counts.items(), key=lambda x: x[1])[0]
            most_freq_mapping = f"{abbr}: {most_freq}"
            writer.writerow([abbr, mapping_str, most_freq_mapping])
    print(f"Processamento concluído: {len(sorted_ambiguous)} abreviações ambíguas foram escritas em '{ambiguous_output_csv}'.")

    # Gera arquivo de palavras únicas: full words associadas a apenas uma abreviação
    # cuja abreviação não seja ambígua (apareça somente com essa full word).
    unique_entries = []
    for full, abbr_counts in full_group.items():
        if len(abbr_counts) == 1:
            # Obtem a única abreviação associada
            abbr = next(iter(abbr_counts))
            # Verifica se essa abreviação não é ambígua
            if abbr in abbr_group and len(abbr_group[abbr]) == 1:
                count = abbr_counts[abbr]
                mapping_str = f"{abbr}: {full}"
                unique_entries.append((full, f"{abbr} ({count})", mapping_str))
    
    sorted_unique = sorted(unique_entries, key=lambda x: x[0])
    
    with open(unique_output_csv, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Full Word", "Abbreviation (Count)", "Mapping"])
        for entry in sorted_unique:
            writer.writerow(entry)
    print(f"Processamento concluído: {len(sorted_unique)} palavras únicas foram escritas em '{unique_output_csv}'.")

if __name__ == "__main__":
    input_csv = "word_by_word_mapping.csv"
    ambiguous_output_csv = "ambiguous_abbreviations_with_counts_filtered.csv"
    unique_output_csv = "unique_full_words.csv"
    process_abbreviations_and_unique(input_csv, ambiguous_output_csv, unique_output_csv)


Processamento concluído: 240 abreviações ambíguas foram escritas em 'ambiguous_abbreviations_with_counts_filtered.csv'.
Processamento concluído: 621 palavras únicas foram escritas em 'unique_full_words.csv'.


In [14]:
import csv

def process_abbreviations_dictionary(input_file: str, output_file: str):
    """
    Lê o arquivo de dicionário de abreviaturas (formato "chave: valor"),
    organiza os registros pelo segundo campo (expansão) e verifica quando há duplicatas.
    Escreve um arquivo de saída com as colunas:
      - Expansion
      - Abbreviations (lista de abreviações associadas, separadas por vírgula)
      - Duplicate? (Yes se houver mais de uma abreviação, No caso contrário)
    """
    expansion_dict = {}
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if not line or ":" not in line:
                continue
            # Separa apenas na primeira ocorrência de ":"
            parts = line.split(":", 1)
            abbr = parts[0].strip()
            expansion = parts[1].strip()
            
            # Agrupa as abreviações pela expansão
            if expansion not in expansion_dict:
                expansion_dict[expansion] = []
            expansion_dict[expansion].append(abbr)
    
    # Ordena as expansões em ordem alfabética
    sorted_expansions = sorted(expansion_dict.items(), key=lambda x: x[0])
    
    # Escreve o resultado num arquivo de saída (usando tabulação como delimitador)
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        writer.writerow(["Expansion", "Abbreviations", "Duplicate?"])
        for expansion, abbr_list in sorted_expansions:
            duplicate_flag = "Yes" if len(abbr_list) > 1 else "No"
            abbr_list_sorted = sorted(abbr_list)
            writer.writerow([expansion, ", ".join(abbr_list_sorted), duplicate_flag])
    
    print(f"Processamento concluído: dados organizados por expansão foram escritos em '{output_file}'.")

if __name__ == "__main__":
    input_file = "abbreviations_dictionary.txt"
    output_file = "duplicates_by_expansion.txt"
    process_abbreviations_dictionary(input_file, output_file)


Processamento concluído: dados organizados por expansão foram escritos em 'duplicates_by_expansion.txt'.


In [22]:
import pandas as pd
import csv

def load_dictionary_from_txt(dictionary_file: str) -> dict:
    """
    Carrega o dicionário de formatos a partir de um arquivo de texto 
    no formato:
       'chave':'valor',
       'outraChave':'outroValor',
    Cada chave é convertida para minúsculas para facilitar a comparação.
    """
    dictionary_data = {}
    with open(dictionary_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.endswith(','):
                line = line[:-1].strip()
            if ':' not in line:
                continue
            
            parts = line.split(':', 1)
            # Remove aspas simples/duplas e espaços
            key = parts[0].strip().strip("'").strip('"').lower()
            value = parts[1].strip().strip("'").strip('"').lower()
            
            dictionary_data[key] = value
    return dictionary_data

def check_expansions_in_excel(excel_file: str, dictionary_data: dict, output_file: str):
    """
    Lê o arquivo Excel (com colunas "Expansion", "Abbreviations" e "Duplicate?"),
    onde "Duplicate?" pode ter valores "Yes" ou "No".
    
    Para cada linha em que "Duplicate?" == "Yes", verifica se "Expansion" 
    (convertido para minúsculas) já existe no dicionário. Se não existir, 
    salva no CSV de saída.
    """
    df = pd.read_excel(excel_file)

    # Colunas do Excel
    expansion_col = "Expansion"
    abbreviations_col = "Abbreviations"
    duplicate_col = "Duplicate?"

    new_terms = []

    for index, row in df.iterrows():
        expansion = str(row[expansion_col]).strip()
        # O campo "Abbreviations" não é essencial para o check,
        # mas poderíamos usar se quiséssemos
        abbreviations = str(row[abbreviations_col]).strip()
        duplicate_decision = str(row[duplicate_col]).strip().lower()  # "yes" ou "no"

        '''if duplicate_decision == "yes":
            expansion_lower = expansion.lower()
            if expansion_lower not in dictionary_data:
                new_terms.append([expansion, abbreviations])'''
        expansion_lower = expansion.lower()
        if expansion_lower not in dictionary_data:
           new_terms.append([expansion, abbreviations])
    # Escreve as novas expansões num arquivo CSV
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile, delimiter=',')
        writer.writerow(["Expansion", "Abbreviations"])
        writer.writerows(new_terms)

    print(f"Foram encontradas {len(new_terms)} expansões que não estão no dicionário.")
    print(f"Elas foram salvas em '{output_file}' para possível inserção futura.")

if __name__ == "__main__":
    # Ajuste os caminhos conforme necessário
    dictionary_file = "formats_dictionary.txt"   # Arquivo no formato 'chave':'valor',
    excel_file = "NovosFormatos.xlsx"           # Nome fictício do seu arquivo
    output_file = "new_expansions_to_add.csv"

    # 1. Carrega o dicionário a partir do arquivo .txt
    dictionary_data = load_dictionary_from_txt(dictionary_file)

    # 2. Verifica as expansões no Excel e grava as novas entradas
    check_expansions_in_excel(excel_file, dictionary_data, output_file)


Foram encontradas 473 expansões que não estão no dicionário.
Elas foram salvas em 'new_expansions_to_add.csv' para possível inserção futura.


In [23]:
import csv

def load_dictionary_from_txt(dictionary_file: str) -> dict:
    """
    Loads a formats dictionary from a text file with lines in the style:
        'someword':'someformat',
        'anotherword':'anotherformat',
    and returns a Python dict with {someword: someformat}.
    Converts both keys and values to lowercase for consistent matching.
    """
    dictionary_data = {}
    with open(dictionary_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Remove trailing comma if present
            if line.endswith(','):
                line = line[:-1].strip()
            # Split only on the first colon
            if ':' not in line:
                continue

            parts = line.split(':', 1)
            word = parts[0].strip().strip("'").strip('"').lower()
            fmt  = parts[1].strip().strip("'").strip('"').lower()
            dictionary_data[word] = fmt
    return dictionary_data

def map_new_words_to_formats(dictionary_data: dict, new_words_file: str, output_file: str):
    """
    Reads a CSV file with new words in the first column,
    checks each one against the dictionary_data, and
    writes an expanded dictionary text file with lines in
    the style 'word':'format', or 'word':'???', if not found.
    """
    new_words = []
    with open(new_words_file, 'r', encoding='utf-8', newline='') as infile:
        reader = csv.reader(infile)
        for row in reader:
            # Skip empty rows
            if not row:
                continue
            # The new word is in the first column
            new_word = row[0].strip()
            new_words.append(new_word)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for word in new_words:
            word_lower = word.lower()
            if word_lower in dictionary_data:
                fmt = dictionary_data[word_lower]
                outfile.write(f"'{word_lower}':'{fmt}',\n")
            else:
                # If not in the dictionary, use ??? as a placeholder
                outfile.write(f"'{word_lower}':'???',\n")

    print(f"Successfully wrote {len(new_words)} lines to '{output_file}'.")

if __name__ == "__main__":
    # 1. Load your existing dictionary of ~1800 words
    dictionary_file = "formats_dictionary.txt"  # Adjust as needed
    dictionary_data = load_dictionary_from_txt(dictionary_file)

    # 2. Read the CSV with the 473 new words
    new_words_file = "new_expansions_to_add.csv"  # The file with your new words in the first column

    # 3. Output file with the final mapping
    output_file = "expanded_dictionary.txt"

    # 4. Map the new words to their formats
    map_new_words_to_formats(dictionary_data, new_words_file, output_file)


Successfully wrote 474 lines to 'expanded_dictionary.txt'.
